In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import os

from scipy import stats
import joblib


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


dataframe = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\predicting-brain-age\Data\original.csv')
fe = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\predicting-brain-age\Data\binned_on_age_original_quartile32.csv')

dataframe = pd.concat([dataframe,fe],axis = 1)

dataframe


In [ ]:
cat_features= dataframe.filter(regex = 'interaction').columns.tolist()

X = dataframe.drop('age', axis=1)
y = (dataframe['age'])

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [ ]:
# cat_features.shape

categorical encoding

In [ ]:
from category_encoders import BinaryEncoder
from sklearn.preprocessing import OneHotEncoder


# Create a BinaryEncoder instance
encoder = BinaryEncoder(cols=cat_features)

# Fit and transform the data
x_train_encoded = encoder.fit_transform(x_train[cat_features])
x_test_encoded = encoder.transform(x_test[cat_features])
# Output the transformed data

x_train = x_train.drop(cat_features,axis = 1)
x_train = pd.concat([x_train,x_train_encoded],axis = 1)

x_test = x_test.drop(cat_features,axis = 1)
x_test = pd.concat([x_test,x_test_encoded],axis = 1)

x_train.shape



In [ ]:
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from lightgbm import LGBMRegressor


space  = [
    Integer(-1, 10, name='max_depth'),
    # Integer(50, 500, name='n_estimators'),
   
    Real(0.5,1,name = 'feature_fraction',prior='log-uniform'),
    Integer(50, 500, name='n_estimators'),

    Integer(20, 200, name='min_data_in_leaf'),
    Real(0.0001, 1, name='learning_rate',prior='log-uniform'),
    Integer(2, 200, name='num_leaves'),
    Real(0.1, 1.0, name='lambda_l1', prior='log-uniform'),  # L1 regularization
    Real(0.1, 1.0, name='wlambda_l2', prior='log-uniform'),  # L2 regularization
    Categorical(['gbdt', 'dart'], name='boosting_type')
]

# Initialize the model
lgbm = LGBMRegressor(objective= 'regression',random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
from skopt import gp_minimize

@use_named_args(space)
def objective(**params):
    lgbm.set_params(**params)
    lgbm.fit(x_train, y_train)
    y_pred = lgbm.predict(x_test)
    mse = mean_absolute_error((y_test), (y_pred))
    return mse

In [ ]:
result = gp_minimize(objective, space, n_calls=50, random_state=42)

print("Best parameters: {}".format(result.x))
print("Best MAE: {:.5f}".format(result.fun))


In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score
from  scipy.stats import pearsonr

lgbm.set_params(**dict(zip([dim.name for dim in space], result.x)))
lgbm.fit(x_train, y_train)
y_pred = lgbm.predict(x_test)
final_mse = mean_absolute_error((y_test), (y_pred))
print(f"Final MAE: {final_mse}")
print('lightgbm r2 :',(r2_score((y_test),(y_pred))))
print('lightgbm pearson :',(pearsonr((y_test),(y_pred))))

In [ ]:
from sklearn.model_selection import cross_val_score
cv = cross_val_score(lgbm,x_train,y_train,scoring='neg_mean_absolute_error',cv=10)
print(np.mean(-cv))